In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
train_df = pd.read_csv('../dataset/RoboMed/train_data.csv')
test_df = pd.read_csv('../dataset/RoboMed/test_data.csv')

In [3]:
train_size = train_df.shape[0]
print(train_size)

283086


In [5]:
train_df = train_df.fillna('')
test_df = test_df.fillna('')

In [6]:
train_df.Revisit.value_counts()

1    234576
0     48510
Name: Revisit, dtype: int64

In [7]:
train_dataframe, valid_dataframe = train_test_split(train_df, test_size=0.1,
                                                    stratify=train_df.Revisit, random_state=42)

In [29]:
def concat_features_ft(df, train=True):
    features = (
        df['Код_диагноза'] + ' | ' +
        df['Диагноз'] + ' | ' +
        'Возраст_' + df['Возраст'].astype(str) + ' | ' +
        'Пол_' + df['Пол'].astype(str) + ' | ' +
        df['Общее состояние'] + ' | ' +
        df['аллергия'] + ' | ' +
        df['Анамнез заболевания'] + ' | ' +
        df['Внешний осмотр']
    )
    
    text = features
    if train:
        text = '__label__' + df.Revisit.astype(str) + ' ' + text

    return text.str.lower().str.replace('\n', '').str.replace(', ', ' , ')

In [30]:
fasttext_train = concat_features_ft(train_dataframe)
fasttext_valid = concat_features_ft(valid_dataframe)
fasttext_test = concat_features_ft(test_df, train=False)

In [32]:
with open('fasttext_train.txt', 'w') as file:
    for line in fasttext_train:
        file.write(line + '\n')

In [33]:
with open('fasttext_valid.txt', 'w') as file:
    for line in fasttext_valid:
        file.write(line + '\n')

In [34]:
with open('fasttext_test.txt', 'w') as file:
    for line in fasttext_test:
        file.write(line + '\n')

In [47]:
! head -n 100 fasttext_valid.txt

__label__1 s91.3 | открытая рана других частей стопы | возраст_31 | пол_2 |  |  | явилась по рез-там обследования | 
__label__1 k30 | диспепсия | возраст_45 | пол_2 |  |  | провел курс назначенного лечения , визит для контрольного осмотра | 
__label__1 m54.3 | ишиас | возраст_36 | пол_2 |  |  | болеет с 2007 балла. травмы , инфекции отрицает , боль связывает с физическими нагрузками. выполнял мрт правого голеностопного сустава деформирующий остеоартроз правого голеностопного сустава. синовит.  принимает артростоп. | 
__label__0 m54.2 | цервикалгия | возраст_63 | пол_1 |  |  | проходит обследование по программе check up.  7 лет принимает 2,5 - 5 мг крестора в связи повышением уровня хс.  адаптирована к цифрам ад 110/70 мм рт ст. болей за грудиной , перебоев в работе сердца не отмечает. регулярно занятия спортом бег на длительные дистанции. | 
__label__1 e28.9 | дисфункция яичников неуточненная | возраст_34 | пол_2 |  |  | выше перечисленные жалобы появились 4 апреля 2016 года в виде рез

In [66]:
! wc fasttext_valid.txt

  28309  682022 8478362 fasttext_valid.txt


In [25]:
! cat fasttext_train.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > fasttext_train_preprocessed.txt

/bin/sh: 1: fasttext: not found


In [ ]:
!  cat fasttext_valid.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > fasttext_valid_preprocessed.txt

In [67]:
!  cat fasttext_test.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > fasttext_test_preprocessed.txt

In [61]:
! fasttext supervised -input fasttext_train.txt -wordNgrams 2 -loss hs -thread 24 -output model_categories

/bin/sh: 1: fasttext: not found


In [ ]:
! fasttext test model_categories.bin fasttext_valid.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_test.txt > y_pred_test.txt

In [ ]:
! fasttext predict-prob model_categories.bin fasttext_valid.txt > y_pred_valid.txt

In [67]:
def read_pred(filename):
    with open(filename) as file:
        lines = file.readlines()
        y_pred = np.array([float(line.split()[1]) if line.split()[0] == '__label__1' else 1 - float(line.split()[1])
                           for line in lines])
    return y_pred

In [68]:
y_pred_valid = read_pred('y_pred_valid.txt')

In [69]:
roc_auc_score(valid_dataframe.Revisit, y_pred_valid)

0.696172647952729

In [70]:
y_pred_test = read_pred('y_pred_test.txt')

In [71]:
submission = pd.read_csv('/root/dataset/RoboMed/sample_submission.csv')

In [72]:
submission.proba = y_pred_test

In [73]:
submission.to_csv('submission_ft.csv', index=False)